In [1]:
!pip install -r requirements.txt --no-warn-conflicts --progress-bar emoji
%load_ext autoreload
%autoreload 2

  Could not find a version that satisfies the requirement urllib2 (from -r requirements.txt (line 4)) (from versions: )
No matching distribution found for urllib2 (from -r requirements.txt (line 4))


In [40]:
import girder_client
import json
import os
import pandas as pd
import psycopg2
import postgres_to_girder
import urllib
from datetime import date

with open ("config.json", "r") as j:
    config = json.load(j)
with open ("context.json", "r") as j:
    context = json.load(j)
api_url = "".join([
    "http://",
    config["girder"]["host"],
    "/api/v1"
])

In [3]:
try:
    gc = girder_client.GirderClient(
        apiUrl=api_url
    )
    gc.authenticate(
        config["girder"]["user"],
        config["girder"]["password"])
    print("Connected to the Girder database 🏗🍃")
except:
    print(
        "I am unable to connect to the "
        "Girder database 🏗🍃"
    )

Connected to the Girder database 🏗🍃


In [4]:
activities_id = postgres_to_girder.get_girder_id_by_name(
    entity="collection",
    name="activities",
    girder_connection=gc
)

In [5]:
try:
    conn = psycopg2.connect(
        " ".join(
            [
                "=".join([
                    key,
                    config["postgres"][key]
                ]) for key in config["postgres"]
            ]
        )
    )
    print("Connected to the Postgres database 🐘")
except:
    print(
        "I am unable to connect to the "
        "Postgres database 🐘"
    )

Connected to the Postgres database 🐘


In [6]:
acts = pd.io.sql.read_sql_query(
    "SELECT * FROM acts;",
    conn
)
users = pd.io.sql.read_sql_query(
    "SELECT * FROM users;",
    conn
)

Pull respondents out of titles in DataFrame from Postgres:

In [7]:
acts["Respondent"] = acts["title"].apply(
    lambda x: x.split(
        " - "
    )[
        1
    ].split(
        " "
    )[
        0
    ] if " - " in x else x.split(
        " – "
    )[
        1
    ].split(
        " "
    )[
        0
    ] if " – " in x else x.split(
        "-"
    )[
        1
    ].split(
        " "
    )[
        0
    ] if "Scale-" in x else x.split(
        " ― "
    )[
        1
    ].split(
        "-"
    )[
        0
    ] if "―" in x else x.split(
        "-"
    )[
        1
    ].split(
        ")"
    )[
        0
    ] if "Index-" in x else "Self" if (
        (
            "_SR" in x
        ) or (
            "-SR" in x
        )
    ) else "Parent" if (
        "_P" in x
    ) else ""
)
acts["title"] = acts["title"].apply(
    lambda x: x.split(
        " - "
    )[
        0
    ] if " - " in x else x.split(
        " – "
    )[
        0
    ] if " – " in x else x.split(
        "-"
    )[
        0
    ] if "Scale-" in x else x.split(
        " ― "
    )[
        0
    ] if "―" in x else x.split(
        "-"
    )[
        0
    ] if "Index-" in x else x.replace(
        " Self Report",
        ""
    ).replace(
        " Parent Report",
        ""
    )
).apply(
    lambda x: "{0})".format(
        x
    ) if (
        "(" in x
    ) and (
        ")"
    ) not in x else x
)

In [8]:
acts

,id,title,type,user_id,act_data,created_at,updated_at,status,organization_id,Respondent
0,1,EHQ (Edinburgh Handedness Questionnaire),survey,1,"{""mode"":""basic"",""frequency"":""1"",""questions"":[{...",2018-02-15 08:36:22.382000-05:00,2018-02-15 08:36:22.382000-05:00,active,None,
1,2,CPIC (Children's Perception of Interparental C...,survey,1,"{""mode"":""basic"",""frequency"":""1"",""questions"":[{...",2018-02-15 08:36:22.646000-05:00,2018-02-15 08:36:22.646000-05:00,active,None,
2,3,APQ_SR (Alabama Parenting Questionnaire),survey,1,"{""mode"":""basic"",""frequency"":""1"",""questions"":[{...",2018-02-15 08:36:22.652000-05:00,2018-02-15 08:36:22.652000-05:00,active,None,Self
3,4,Daily Diary Card (CMI DBT Diary Card),survey,3,"{""mode"":""basic"",""frequency"":""1"",""questions"":[{...",2018-02-19 15:07:49.975000-05:00,2018-02-19 15:07:49.975000-05:00,active,None,
4,5,Daily Diary Card (CMI DBT Diary Card),survey,3,"{""mode"":""basic"",""frequency"":""1"",""questions"":[{...",2018-02-19 15:07:54.157000-05:00,2018-02-19 15:07:54.157000-05:00,active,None,
5,6,Daily Diary Card (CMI DBT Diary Card),survey,3,"{""mode"":""basic"",""frequency"":""1"",""questions"":[{...",2018-02-19 17:18:18.046000-05:00,2018-02-19 17:18:18.046000-05:00,active,None,
6,33,Good things,voice,6,"{""frequency"":""1d"",""timer"":0,""instruction"":""Tal...",2018-02-21 17:51:11.768000-05:00,2018-02-21 17:51:46.780000-05:00,active,None,
7,31,What's he looking at?,drawing,6,"{""frequency"":""1d"",""timer"":10,""image_url"":""http...",2018-02-21 17:49:21.721000-05:00,2018-02-21 17:53:31.092000-05:00,active,None,
8,10,Example survey,survey,6,"{""questions"":[{""type"":""multi_sel"",""rows"":[{""te...",2018-02-21 12:00:58.491000-05:00,2018-02-21 12:08:48.687000-05:00,active,None,
9,16,PAQ_C (Physical Activity Questionnaire for Old...,survey,3,"{""mode"":""basic"",""frequency"":""1"",""questions"":[{...",2018-02-21 17:48:20.234000-05:00,2018-02-21 17:48:20.234000-05:00,active,None,


In [37]:
for i, activity in enumerate(
    list(
        acts["title"]
    )
):
    abbreviation = None
    if "(" in activity:
        anames = [
            a.strip(
                ")"
            ).strip() for a in activity.split(
                "("
            )
        ]
        if (
            len(anames)==2
        ):
            if (
                len(anames[0])>len(anames[1])
            ):
                abbreviation = anames[1]
                activity_name = anames[0]
            else:
                abbreviation = anames[0]
                activity_name = anames[1]
        else:
            print(anames)
    activity_name = activity if not abbreviation else activity_name
            
    respondent = acts[
        acts["title"]==activity
    ]["Respondent"].values[0]
    item_version = "{0} ({1})".format(
        " ― ".join([
            "Healthy Brain Network",
            "{0} ({1}) {2}".format(
                activity_name,
                abbreviation,
                "― {0} Report".format(
                    respondent
                ) if len(respondent) else ""
            ) if abbreviation else "{0} {1}".format(
                activity,
                "― {0} Report".format(
                    respondent
                ) if len(respondent) else ""
            )
        ]).strip(" "),
        date.strftime(
            acts.loc[
                i,
                "updated_at"
            ],
            "%F"
        )
    )
        
    # Create or locate top-level folder and return _id
    activity_folder_id = gc.createFolder(
        name=activity if not abbreviation else activity_name,
        parentId=activities_id,
        parentType="collection",
        public=True,
        reuseExisting=True
    )["_id"]
    
    # Create or locate item
    act_data = json.loads(
        acts.loc[
            i,
            "act_data"
        ]
    )
    
    
    activity_item_id = gc.createItem(
        name=item_version,
        parentFolderId=activity_folder_id,
        reuseExisting=True,
        metadata={
            **context,
            "schema:name": {
                "@value": activity_name,
                "@language": "en-US"
            },
            "abbreviation": abbreviation if abbreviation else None,
            "@type": acts.loc[
                i,
                "type"
            ],
            "status": acts.loc[
                i,
                "status"
            ],
            "pav:lastUpdatedOn": acts.loc[
                i,
                "updated_at"
            ].isoformat(),
            **{ prop: act_data[prop] for prop in 
                act_data if prop not in [
                    "questions",
                    "instruction",
                    "image_url"
                ]
            },
            "instruction": {
                "@value": act_data["instruction"],
                "@language": "en-US"
            } if (
                (
                    "instruction" in act_data
                ) and len(
                    act_data["instruction"]
                )
            ) else None
        }
    )["_id"]
        
    # Upload applicable file(s)
    
    img = urllib.request.urlopen(
        act_data["image_url"]
    ) if "image_url" in act_data else None
    if img:
        new_img_id = gc.uploadFile(
            parentId=activity_item_id,
            stream=img,
            name=".".join([
                ''.join([
                    c for c in activity_name if c.isalnum()
                ]),
                act_data[
                    "image_url"
                ].split("?")[0].split(".")[-1]
            ]),
            size=int(img.info()["Content-Length"])
        )["_id"]
        gc.addMetadataToItem(
            itemId=activity_item_id,
            metadata={
                "image_url": "".join([
                    api_url,
                    "/file/",
                    new_img_id,
                    "/download?contentDisposition=inline"
                ])
            }
        )

In [38]:
users

,id,first_name,last_name,email,password,role,newsletter,verify_token,access_token,created_at,updated_at,status,organization_id
0,2,erik tester,None,erik@test.com,$2a$10$9S9QcZgV.HaS.nRJyKIMmOaa6KN1mSthENxRSZy...,user,True,None,1525288601912_ALrxunzLI68lHtsA,2018-02-15 08:24:30.088000-05:00,2018-05-02 15:16:41.915000-04:00,active,None
1,4,Erik P,None,Erik2@test.com,$2a$10$6Z4.zKbVDuJQgKY/bPVGgewqu2V.gXvTVdq9/ai...,admin,True,None,1526041082535_!BP1pIBJxqRcrtSc,2018-02-20 22:54:54.107000-05:00,2018-05-11 08:18:02.538000-04:00,active,None
2,3,Super Admin,,admin@mindlogger.org,$2a$10$uSf8MUacb2SAPsTUAk5iWeFZqSz5lbsc.BqhZwQ...,super_admin,None,None,1526412592448_pDhwqdrbPvH1EGI5,2018-02-19 12:02:31.396000-05:00,2018-05-15 15:29:52.452000-04:00,active,None
3,7,Binary,Bottle,arno@binarybottle.com,$2a$10$r8Aq8I9CgMxoqTtpnRaOkODPicij/iXprZBN9ls...,viewer,None,None,None,2018-03-05 08:22:18.269000-05:00,2018-03-05 08:22:18.269000-05:00,active,None
4,6,Arno,Klein,arno@childmind.org,$2a$10$BlYb6oHohZ7O4XhVtIh.TuOP1hjBTBdZOe8FUQs...,user,None,None,1519331037257_BsSKwkTCXChF!AfM,2018-02-21 11:37:43.657000-05:00,2018-02-22 15:23:57.261000-05:00,active,None
5,8,Mind,Boggle,arno@mindboggle.info,$2a$10$T4VzPiSAn4SigRUIqOrmO.6Z0JrYOH2n53ndUgl...,user,None,None,None,2018-03-05 08:30:52.483000-05:00,2018-03-05 08:30:52.483000-05:00,active,None
6,1,Admin,,admin@ab2cd.com,$2a$10$MmTz3hdl.YqystN/mj7DtuTjzcrTgPv.i7S8NFE...,user,None,None,,2018-02-14 04:33:51.835000-05:00,2018-03-05 10:51:17.608000-05:00,active,None
7,5,Jon Clucas,None,jon.clucas@chilldmind.org,$2a$10$TQE.ecRlvJG3X0yNsRpjPOv3uL01g/jrxWDQ3UI...,user,True,None,1519323782601_17P6dtlkynPKNRZe,2018-02-21 10:08:16.461000-05:00,2018-02-22 13:23:10.941000-05:00,active,None
8,9,Jon,Clucas,jon@vasegurt.com,$2a$10$CTmMp3pZjfgyQk2iwWs7iO.oERu8UwTnTZxJVXN...,user,None,None,1522953391673_fgP4ozZxGwJreuaz,2018-04-05 14:25:54.403000-04:00,2018-04-05 14:36:31.676000-04:00,active,None
9,10,None,None,anirudh.krishnakumar@childmind.org,$2a$10$4ENAa80r8C4lX0.XNSfGLuMuW3VmWOhimR9CMcS...,None,None,None,,2018-05-01 13:43:05.864000-04:00,2018-05-01 17:33:44.918000-04:00,active,None


In [54]:
get_user_id_by_email(gc, "test@example.com")